# Project Assignment #1: Confidence Intervals and Hypothesis Testing
## Team Member:
- Alex Magnus
- Erkhes Tumurtogoo
- Jianyuan Qiu
- Rithvik Sunil
- Haoliang Zhao

In [1]:
import pandas as pd
import numpy as np
df_indicators = pd.read_csv('country_indicators.csv')
df_preds = pd.read_csv('test_predictions.csv')
df_merged = df_preds.merge(df_indicators, left_on='iso3', right_on='iso3', how='inner')

## "Unpaired" Data
- Make sure the sequence of analyses that you perform are clear and understandable. E.g., what model(s) data are used, and for what data subset(s)?

    - Your mark for the Tutorial Assignment will be based on completing the items below and the clarity and correctness of your work.

### 1. Create the Prediciton Probability "Error" results for the xgboost and ffnn models analagously to the transformer model produced above. - **Alex**

In [2]:
import numpy as np
#TRANSFORMER
df_preds['transformer_probability_prediction_error'] = np.abs(df_preds['y_true_transformer'].astype(float) - df_preds['y_pred_proba_transformer'])

#XGboost
df_preds['xgboost_probability_prediction_error'] = np.abs(df_preds['y_true_xgboost'].astype(float) - df_preds['y_pred_proba_xgboost'])

#ffnn
df_preds['ffnn_probability_prediction_error'] = np.abs(df_preds['y_true_ffnn'].astype(float) - df_preds['y_pred_proba_ffnn'])

df_preds[['y_true_transformer','y_pred_proba_transformer','transformer_probability_prediction_error', 'xgboost_probability_prediction_error', 'ffnn_probability_prediction_error']]

,y_true_transformer,y_pred_proba_transformer,transformer_probability_prediction_error,xgboost_probability_prediction_error,ffnn_probability_prediction_error
0,False,0.183897,0.183897,0.066500,0.409958
1,False,0.267831,0.267831,0.099643,0.406696
2,False,0.482585,0.482585,0.295914,0.545236
3,False,0.187792,0.187792,0.361556,0.534560
4,True,0.539319,0.460681,0.608380,0.461417
...,...,...,...,...,...
359,False,0.182196,0.182196,0.079453,0.291874
360,False,0.203236,0.203236,0.060189,0.300321
361,False,0.527107,0.527107,0.302375,0.335496
362,False,0.555677,0.555677,0.729246,0.324000


### 2. Create a bootsrap confidence interval for the average Prediction Probability "Error" for one of these models using all the data. - **Alex**
> In this context "using all the data" means using all the predictions made for a given model under consideration; whereas, "using a data subset" would mean restricting the rows of the data on the basis of one (or more) of the columns from the "Progress Indicators" data so as to only consider the predictions made for the given model under consideration within a given subset of (the countries of) the data.

### Confidence Interval with 95%

In [3]:
reps = 1000
average_error_of_transformer = np.zeros(reps)
np.random.seed(123)

for rep in range(reps):
    bootstrap_sample = np.random.choice(df_preds['transformer_probability_prediction_error'], size=df_preds.shape[0])
    average_error_of_transformer[rep] = bootstrap_sample.mean()

np.quantile(average_error_of_transformer, (0.025, 0.975))

array([0.42372167, 0.45860283])

### 3. Create the Prediction Classification "Correctness" results of "correct" and "incorrect" predictions for the `transformer`, `xgboost` and `ffnn` models; or, an alternative "either/or" breakdown of interest (such as "wrongly predicted no escalation" versus all the other categories combined). - **Alex**

In [4]:
# Prediction Classification "Correctness"

threshold_transformer = 0.5
threshold_xgboost = 0.5
threshold_ffnn = 0.5

df_preds['transformer_classifcation_performance_outcome'] = None
df_preds['xgboost_classifcation_performance_outcome'] = None
df_preds['ffnn_classifcation_performance_outcome'] = None

tmp = df_preds['transformer_classifcation_performance_outcome'].copy()
#tmp_xgboost = df_preds['xgboost_classifcation_performance_outcome'].copy()
#tmp_ffnn = df_preds['ffnn_classifcation_performance_outcome'].copy()

#TRANSFORMER
TP_pos_pred_correct = df_preds.y_true_transformer & (df_preds.y_pred_proba_transformer>threshold_transformer)
tmp[TP_pos_pred_correct] = "correctly predicted escalation"
TN_neg_pred_correct = (~df_preds.y_true_transformer) & (df_preds.y_pred_proba_transformer<=threshold_transformer)
tmp[TN_neg_pred_correct] = "correctly predicted no escalation"
FP_pos_pred_wrong = (~df_preds.y_true_transformer) & (df_preds.y_pred_proba_transformer>threshold_transformer)
tmp[FP_pos_pred_wrong] = "wrongly predicted escalation"
FN_neg_pred_wrong = df_preds.y_true_transformer & (df_preds.y_pred_proba_transformer<=threshold_transformer)
tmp[FN_neg_pred_wrong] = "wrongly predicted no escalation"

df_preds['transformer_classifcation_performance_outcome'] = tmp

#xgboost
TP_pos_pred_correct = df_preds.y_true_xgboost & (df_preds.y_pred_proba_xgboost>threshold_xgboost)
tmp[TP_pos_pred_correct] = "correctly predicted escalation"
TN_neg_pred_correct = (~df_preds.y_true_xgboost) & (df_preds.y_pred_proba_xgboost<=threshold_xgboost)
tmp[TN_neg_pred_correct] = "correctly predicted no escalation"
FP_pos_pred_wrong = (~df_preds.y_true_xgboost) & (df_preds.y_pred_proba_xgboost>threshold_xgboost)
tmp[FP_pos_pred_wrong] = "wrongly predicted escalation"
FN_neg_pred_wrong = df_preds.y_true_xgboost & (df_preds.y_pred_proba_xgboost<=threshold_xgboost)
tmp[FN_neg_pred_wrong] = "wrongly predicted no escalation"

df_preds['xgboost_classifcation_performance_outcome'] = tmp

#ffnn
TP_pos_pred_correct = df_preds.y_true_ffnn & (df_preds.y_pred_proba_ffnn>threshold_ffnn)
tmp[TP_pos_pred_correct] = "correctly predicted escalation"
TN_neg_pred_correct = (~df_preds.y_true_ffnn) & (df_preds.y_pred_proba_ffnn<=threshold_ffnn)
tmp[TN_neg_pred_correct] = "correctly predicted no escalation"
FP_pos_pred_wrong = (~df_preds.y_true_ffnn) & (df_preds.y_pred_proba_ffnn>threshold_ffnn)
tmp[FP_pos_pred_wrong] = "wrongly predicted escalation"
FN_neg_pred_wrong = df_preds.y_true_ffnn & (df_preds.y_pred_proba_ffnn<=threshold_ffnn)
tmp[FN_neg_pred_wrong] = "wrongly predicted no escalation"

df_preds['ffnn_classifcation_performance_outcome'] = tmp

df_preds[['y_true_transformer','y_pred_transformer','transformer_classifcation_performance_outcome']]

,y_true_transformer,y_pred_transformer,transformer_classifcation_performance_outcome
0,False,False,correctly predicted no escalation
1,False,False,correctly predicted no escalation
2,False,False,correctly predicted no escalation
3,False,False,correctly predicted no escalation
4,True,True,correctly predicted escalation
...,...,...,...
359,False,False,correctly predicted no escalation
360,False,False,correctly predicted no escalation
361,False,True,wrongly predicted escalation
362,False,True,wrongly predicted escalation


### 4. Perform a one sample hypothesis test of the proportion of a specific Prediction Classification "Correctness" category for another of these models using all the data. - **Haoliang**
- When performing a hypothesis test you'll need to determine and specify the null hypothesis under consideration; obtain a p-value (either through simulation, or `scipy.stats.binom` or `scipy.stats.ttest_1samp`); and, finally, provide a statement of the degree of evidence against the null hypothesis in the usual manner (of https://www.jcpcarchives.org/userfiles/values-of-p-Inference.jpg).

### Null Hypothesis
$H_\text{trans_0}: p_\text{preds}=p=0.5 (\text{The proportion of the Transformer Prediction Classification "Correctness" category is 0.5})$

$H_\text{xgboo_0}: p_\text{preds}=p=0.5 (\text{The proportion of the Xgboost Prediction Classification "Correctness" category is 0.5})$

$H_\text{ffnn_0}: p_\text{preds}=p=0.5 (\text{The proportion of the FFNN Prediction Classification "Correctness" category is 0.5})$

In [5]:
np.random.seed(123)
null = 0.5
rep,n=10000,df_preds.shape[0]

# For Transformer
observed_test_stat =  \
((df_preds['transformer_classifcation_performance_outcome'] == 'correctly predicted no escalation')
 | (df_preds['transformer_classifcation_performance_outcome'] == 'correctly predicted escalation')).mean()

test_stat = np.zeros(rep)
for i in range(rep):
    sample_mean = sum(np.random.choice(['Correct', 'Incorrect'], size=n) == 'Correct')/n
    test_stat[i] = sample_mean

sum(abs(test_stat-null) >= abs(observed_test_stat-null)) / rep # p-value=0

0.0

In [6]:
# For Xgboost
np.random.seed(123)
observed_test_stat =  \
((df_preds['xgboost_classifcation_performance_outcome'] == 'correctly predicted no escalation')
 | (df_preds['xgboost_classifcation_performance_outcome'] == 'correctly predicted escalation')).mean()

test_stat = np.zeros(rep)
for i in range(rep):
    sample_mean = sum(np.random.choice(['Correct', 'Incorrect'], size=n) == 'Correct')/n
    test_stat[i] = sample_mean

sum(abs(test_stat-null) >= abs(observed_test_stat-null)) / rep # p-value=0.8839

0.8782

In [7]:
# For ffnn
np.random.seed(123)
observed_test_stat =  \
((df_preds['ffnn_classifcation_performance_outcome'] == 'correctly predicted no escalation')
 | (df_preds['ffnn_classifcation_performance_outcome'] == 'correctly predicted escalation')).mean()

test_stat = np.zeros(rep)
for i in range(rep):
    sample_mean = sum(np.random.choice(['Correct', 'Incorrect'], size=n) == 'Correct')/n
    test_stat[i] = sample_mean

sum(abs(test_stat-null) >= abs(observed_test_stat-null)) / rep #p-value=0

0.0

### Conclusion:
By comparing the p-value to the table, We conclude that there is strong evidence against our null hypothesis of Transformer and FFNN since the p-values for both of them are 0. Still, on the other hand, there is no evidence against our null hypothesis of Xgboost since the p-value is 0.8692 and since its p-value are close to the 1, the proportion of our null hypoyhesis is close to the real proportion of Xgboost.

### 5. Consider the "Progress Indicators" data and use "boolean selection" with one (or more) of the columns to restrict the data to a subset (of rows) of data and repeat either of the (confidence interval and hypothesis testing) analyses above but this time instead only using this specified subset of countries. - **Haoliang**

    - *Potentially relevant subsets of data that might be of interest could be created on the basis of Human Development Index categories, Fragile States Index categories, World Bank economy categories, etc. (e.g., `'fsi_category', 'hdr_hdicode', 'hdr_region', 'wbi_income_group', 'wbi_lending_category', 'wbi_other_(emu_or_hipc)'`, etc.); or, perhaps by alternative boolean selections based on restricting the data to countries with specific continuous variable values that fall within specified thresholds or limits.*

### In the 'Boolean selection', we decided to select all countries with the state of fsi_category as 'Warning', and we redo both Confidence Interval and Hypothesis tests.

### Confidence Interval with 95%

In [8]:
condition_lst = df_indicators[df_indicators.fsi_category == 'Warning']['iso3']
reps = 1000
average_error = np.zeros(reps)
np.random.seed(123)

## For Transfomer
for rep in range(reps):
    bootstrap_sample = np.random.choice(df_preds[df_preds['iso3'].isin(condition_lst)]['transformer_probability_prediction_error'],
                                        size=df_preds[df_preds['iso3'].isin(condition_lst)].shape[0])
    average_error[rep] = bootstrap_sample.mean()

np.round(np.quantile(average_error, (0.025, 0.975)), 3) # [0.455, 0.504]

array([0.455, 0.504])

In [9]:
## For Xgboost
for rep in range(reps):
    bootstrap_sample = np.random.choice(df_preds[df_preds['iso3'].isin(condition_lst)]['xgboost_probability_prediction_error'],
                                        size=df_preds[df_preds['iso3'].isin(condition_lst)].shape[0])
    average_error[rep] = bootstrap_sample.mean()

np.round(np.quantile(average_error, (0.025, 0.975)), 3) # [0.49 , 0.528]

array([0.482, 0.537])

In [10]:
## For FFNN
for rep in range(reps):
    bootstrap_sample = np.random.choice(df_preds[df_preds['iso3'].isin(condition_lst)]['ffnn_probability_prediction_error'],
                                        size=df_preds[df_preds['iso3'].isin(condition_lst)].shape[0])
    average_error[rep] = bootstrap_sample.mean()

np.round(np.quantile(average_error, (0.025, 0.975)), 3) # [0.415 , 0.444]

array([0.415, 0.443])

### Redo the Hypothesis test

In [11]:
df_preds[df_preds['iso3'].isin(condition_lst)].shape[0]

171

In [12]:
np.random.seed(123)
condition_lst = df_indicators[df_indicators.fsi_category == 'Warning']['iso3']
null = 0.5
rep = 10000

## For Transfomer
observed_test_stat =  \
((df_preds[df_preds['iso3'].isin(condition_lst)]['transformer_classifcation_performance_outcome'] == 'correctly predicted no escalation')
 | (df_preds[df_preds['iso3'].isin(condition_lst)]['transformer_classifcation_performance_outcome'] == 'correctly predicted escalation')).mean()

test_stat = np.zeros(rep)
n = df_preds[df_preds['iso3'].isin(condition_lst)].shape[0]
for i in range(rep):
    sample_mean = sum(np.random.choice(['Correct', 'Incorrect'], size=n) == 'Correct')/n
    test_stat[i] = sample_mean

sum(abs(test_stat-null) >= abs(observed_test_stat-null)) / rep # p-value=0.1268

0.1268

In [13]:
## For Xgboost
np.random.seed(123)
observed_test_stat =  \
((df_preds[df_preds['iso3'].isin(condition_lst)]['xgboost_classifcation_performance_outcome'] == 'correctly predicted no escalation')
 | (df_preds[df_preds['iso3'].isin(condition_lst)]['xgboost_classifcation_performance_outcome'] == 'correctly predicted escalation')).mean()

test_stat = np.zeros(rep)
n = df_preds[df_preds['iso3'].isin(condition_lst)].shape[0]
for i in range(rep):
    sample_mean = (np.random.choice(['Correct', 'Incorrect'], size=n) == 'Correct').mean()
    test_stat[i] = sample_mean

sum(abs(test_stat-null) >= abs(observed_test_stat-null)) / rep # p-value=0.0095

0.0095

In [14]:
## For FFNN
np.random.seed(123)
observed_test_stat =  \
((df_preds[df_preds['iso3'].isin(condition_lst)]['ffnn_classifcation_performance_outcome'] == 'correctly predicted no escalation')
 | (df_preds[df_preds['iso3'].isin(condition_lst)]['ffnn_classifcation_performance_outcome'] == 'correctly predicted escalation')).mean()

test_stat = np.zeros(rep)
n = df_preds[df_preds['iso3'].isin(condition_lst)].shape[0]
for i in range(rep):
    sample_mean = sum(np.random.choice(['Correct', 'Incorrect'], size=n) == 'Correct')/n
    test_stat[i] = sample_mean

sum(abs(test_stat-null) >= abs(observed_test_stat-null)) / rep # p-value=0.0

0.0

### Conclusion:
For the average of probability_prediction_error with the country statement as 'Warning'

there is a 95% chance that the true parameter falls between 0.455 and 0.504 for Transformer,

there is a 95% chance that the true parameter falls between 0.481 and 0.537 for Xgboost,

there is a 95% chance that the true parameter falls between 0.415 and 0.444 for FFNN;

By comparing the p-value to the table, we conclude that there is strong evidence against our null hypothesis of FFNN since the p-value is 0. There is good evidence against our null hypothesis of Xgboost since the p-value is 0.0095. Still, on the other hand, there is no evidence against our null hypothesis of Transformer since the p-value is 0.1268 and

### 6. (and 7.) Create a two-sample bootstrap confidence interval and perform a hypothesis test comparing the performance of a single model for the data subset created above versus the remaining data not included in that data subset. - **Haoliang**
  - A two-sample bootstrap confidence interval is created by repeatedly creating a "'pair" of bootstrapped samples" by making a single bootstrap sample for samples two data subsets individually, and then the other, and then those two "single sample" bootstrapped samples together make up the "'pair" of bootrapped samples".
  <br><br>
    
  - A hypothesis test for two ("unpaired") samples can be carried out on the basis of permutating shuffling group membership (while ensuring that the original subset sample sizes remain unchanged) in order to create a sampling distribution under a null hypothesis assumption of "no difference between groups", or based on `scipy.stats.median_test` which assumes the *medians* of the two groups are identical (or the more powerful `scipy.stats.mannwhitneyu` which again assumes "no difference between groups"), or `scipy.stats.ttest_ind` which assumes the *means* of the two groups are identical (and that the samples come from normally distributed populations).

### We decided to do the comparison for Transformer,

### with $H_0: \text{There is no difference between those two groups}$

### For two-sample bootstrapped CI with 95%

In [15]:
np.random.seed(123)
condition_lst = df_indicators[df_indicators.fsi_category == 'Warning']['iso3']
reps = 1000

average_error_difference = np.zeros(reps)
for rep in range(reps):
    bootstrap_sample1 = np.random.choice(df_preds[df_preds['iso3'].isin(condition_lst)]['transformer_probability_prediction_error'],
                                        size=df_preds['iso3'].isin(condition_lst).shape[0])
    bootstrap_sample2 = np.random.choice(df_preds[~df_preds['iso3'].isin(condition_lst)]['transformer_probability_prediction_error'],
                                        size=df_preds[~df_preds['iso3'].isin(condition_lst)].shape[0])

    average_error_difference[rep] = bootstrap_sample1.mean() - bootstrap_sample2.mean()

np.round(np.quantile(average_error_difference, [0.025, 0.975]), 3) # [0.043,  0.103]

array([0.043, 0.103])

### For Null Hypothesis Test

In [16]:
from scipy import stats
stats.mannwhitneyu(df_preds[df_preds['iso3'].isin(condition_lst)]['transformer_probability_prediction_error'],
                  df_preds[~df_preds['iso3'].isin(condition_lst)]['transformer_probability_prediction_error']) # p-value=0.00024

MannwhitneyuResult(statistic=20182.0, pvalue=0.0002397641612548599)

### Conclusion:
For the different average of probability_prediction_error with the country statement as 'Warning' and not 'Warning', there is a 95% chance that the true parameter of the differences between the two  falls between 0.043 and 0.103 for Transformer. Meaning that the transformer model is more accurate for countries that are not in the Warning category.

By comparing the p-value to the table, We conclude that there is strong evidence against our null hypothesis since the p-value is 0.00024.

## "Paired" Data

The analyses above considered one- and two-sample subsets of data, with samples defined on the basis of specific subsets of data; however, this data utilized above is considered "unpaired" since a given *individual* observed outcome is not considered "twice" in a "repeated" or "paired" sort of way with respect to some slightly different conditions. This is no longer the case if we instead compare the result of predictions from two different models on some (sub)set of data; because, since predictions from the two different models are made for the same country, these individual predictions (on the same country) can be natually "paired" together.  

It turns out that "paired" data is naturally more powerful than "unpaired" data because it allows us to examine the comparision on the basis of the behavior of the pairs of data, as opposed to a relative comparison between one whole sample versus another whole sample.  To perform a "paired" analysis, each individual "pair" of data is turned into a single numeric value of the difference (calculated in the same consistent manner across all sets of pairs) between the values of the pair. So for two samples $x$ and $y$ where $x_i$ is "paired" with $y_i$, the "paired" analysis simply becomes a one-sample anslysis on the basis of $z_i=x_i-y_i$ with a natural null hypothesis assumption that there is no difference on average between the two values comprising the pairs.

> Take care to note that paring two samples requires more than just having the same sample size: the pairing must reflect an actual naturally meaningful pair construction justification, such as two prediction made by different models but on the same country.


### 8. (and 9.) Create a bootstrap confidence interval and a hypothesis test comparing the performance of two the models across all the data on the basis of a "paired" sample analysis (by transforming the paired sample into a single $z_i=x_i-y_i$ difference sample). - **Rithvik**
    - A bootstrap confidence interval is created by bootrapping from the sample of "paired differences"; whereas, the sampling distribution of the null hypothesis of "the group an observation belongs to doesn't matter" can be constructed using a permutation shuffling approach which randomly reassigns the sample membership within each of the paired samples. Functions performing theoretical nonparametric and parametric "paired" sample analyses are `scipy.stats.wilcoxon` and `scipy.stats.ttest_rel`, where the null hypothesis of the former assumes the slightly different "no tendency for one of the samples in the pair to be larger than the other", while the null hypothesis of the latter assumes "no difference on average" between the pairs (and that the samples come from normally distributed populations).

Creating a bootstrap CI 95%

In [17]:
np.random.seed(123)

reps = 1000
bootstrapped_difference_between_two_models = np.zeros(reps)

# transformer
#sample1 = np.random.choice(df_preds[df_preds['iso3'].isin(condition_lst)]['transformer_probability_prediction_error'],
                                        #size=df_preds.shape[0])
# xgboost
#sample2 = np.random.choice(df_preds[df_preds['iso3'].isin(condition_lst)]['xgboost_probability_prediction_error'],
                                        #size=df_preds.shape[0])

sample_difference = df_preds[df_preds['iso3'].isin(condition_lst)]['transformer_probability_prediction_error'] - df_preds[df_preds['iso3'].isin(condition_lst)]['xgboost_probability_prediction_error']


for rep in range(reps):
  bootstrap_sample = np.random.choice(sample_difference, size=df_preds['iso3'].isin(condition_lst).shape[0])
  bootstrapped_difference_between_two_models[rep] = bootstrap_sample.mean()

np.round(np.quantile(bootstrapped_difference_between_two_models, [0.025, 0.975]), 3)

array([-0.052, -0.008])

In [18]:
stats.mannwhitneyu(df_preds[df_preds['iso3'].isin(condition_lst)]['transformer_probability_prediction_error'],
                   df_preds[df_preds['iso3'].isin(condition_lst)]['xgboost_probability_prediction_error']) #pvalue [0.029]

MannwhitneyuResult(statistic=12627.0, pvalue=0.029258109837566448)


### 10. Repeat the above analyses for different model pairs on some different subsets of data. - **Alex**

    - **Ideally, you will use this exercise to identify different subsets of the data where there's demonstrable statistical evidence that the different predictive models have differential performance levels. These would then be provide different characterizations on the basis of the "Progress Indicators" data as to differential performance between the different (`xgboost`, `ffnn`, and `transformer`) predictive models. Finding such potential explanations for differential predictive model performance has the potential to form the basis of your final project slides and presentation, and can help make completing your project a relatively straightfoward task that essentially only requires subsequent reanalysis using a linear regression model and chracterizations elaborating on the findings you've already made based on this analysis here.**

    > *Potentially relevant subsets of data that might be of interest could be created on the basis of Human Development Index categories, Fragile States Index categories, World Bank economy categories, etc. (e.g., `'fsi_category', 'hdr_hdicode', 'hdr_region', 'wbi_income_group', 'wbi_lending_category', 'wbi_other_(emu_or_hipc)'`, etc.); or, perhaps by alternative boolean selections based on restricting the data to countries with specific continuous variable values that fall within specified thresholds or limits.*

    - For the purposes of **Project Assignment #1** it's not mandatory that your project team has managed to produce statistical evidence suggestive of differences in the performance of the three models (`xgboost`, `ffnn`, and `transformer`) within different country subsets; however, you will need to do so in order to adequately complete **Project Assignment #2** which is due on Nov 13 (the Monday after you return from Reading Week), so hopefully you'll be able to make some progress towards this task through your work and submission for **Project Assignment #1**.
    
        > A reasonable approach to identifying model performances differences within different country subsets is to first explore model performance **matrics** in the manner of the Oct 13 **Project Tutorial Activity and Assignment**, and then subsequently apply confidence interval and/or hypothesis testing methodologies in terms of the Prediction Probability "Error" or Prediciton Classification "Correctness" outcomes considered here in **Project Assignment #1** in order to provide statistical evidence of any observed model performance differences within those identified country subsets.
        

- Make sure the sequence of analyses that you perform are clear and understandable. E.g., what model(s) data are used, and for what data subset(s)?
    - **Your mark for the Tutorial Assignment will be based on completing the items below and the clarity and correctness of your work.**

# THIS IS THE CODE TO FIND LOWEST P_VALUES

In [19]:
import numpy as np
#TRANSFORMER
df_merged['transformer_probability_prediction_error'] = np.abs(df_preds['y_true_transformer'].astype(float) - df_preds['y_pred_proba_transformer'])

#XGboost
df_merged['xgboost_probability_prediction_error'] = np.abs(df_preds['y_true_xgboost'].astype(float) - df_preds['y_pred_proba_xgboost'])

#ffnn
df_merged['ffnn_probability_prediction_error'] = np.abs(df_preds['y_true_ffnn'].astype(float) - df_preds['y_pred_proba_ffnn'])

In [20]:
df_merged_copy = df_merged.copy()

In [21]:
print(df_merged_copy.isnull().sum().sort_values())

yearmonth                                                                                                                  0
sowc_women-s-economic-empowerment__unemployment-rate-2010-2020-r_male_rural                                                0
sowc_women-s-economic-empowerment__labour-force-participation-rate-2010-2020-r_female_total                                0
sowc_women-s-economic-empowerment__labour-force-participation-rate-2010-2020-r_female_urban                                0
sowc_women-s-economic-empowerment__labour-force-participation-rate-2010-2020-r_female_rural                                0
                                                                                                                        ... 
sowc_children-with-disabilities__education_foundational-learning-skills-2017-2021-r_children-without-disabilities        316
sowc_hiv-aids-intervention-coverage__condom-use-among-adolescents-age-15-19-with-multiple-partners-2012-2020-r_female    316


In [22]:
df_merged

,yearmonth,fips,y_pred_transformer,y_pred_proba_transformer,y_true_transformer,y_pred_xgboost,y_pred_proba_xgboost,y_true_xgboost,y_pred_ffnn,y_pred_proba_ffnn,...,fsi_p1:_state_legitimacy,fsi_p2:_public_services,fsi_p3:_human_rights,fsi_c1:_security_apparatus,fsi_c2:_factionalized_elites,fsi_x1:_external_intervention,fsi_category,transformer_probability_prediction_error,xgboost_probability_prediction_error,ffnn_probability_prediction_error
0,202211,FJ,False,0.183897,False,False,0.066500,False,False,0.409958,...,6.1,4.2,5.4,6.4,8.2,6.6,Warning,0.183897,0.066500,0.409958
1,202212,FJ,False,0.267831,False,False,0.099643,False,False,0.406696,...,6.1,4.2,5.4,6.4,8.2,6.6,Warning,0.267831,0.099643,0.406696
2,202211,TZ,False,0.482585,False,True,0.704086,True,True,0.545236,...,6.9,8.4,5.6,4.6,6.5,6.0,Warning,0.482585,0.295914,0.545236
3,202212,TZ,False,0.187792,False,True,0.638444,True,True,0.534560,...,6.9,8.4,5.6,4.6,6.5,6.0,Warning,0.187792,0.361556,0.534560
4,202301,TZ,True,0.539319,True,True,0.608380,False,True,0.538583,...,6.9,8.4,5.6,4.6,6.5,6.0,Warning,0.460681,0.608380,0.461417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,202211,MJ,False,0.182196,False,False,0.079453,False,False,0.291874,...,3.9,3.8,2.9,4.0,6.5,6.3,Stable,0.182196,0.079453,0.291874
360,202212,MJ,False,0.203236,False,False,0.060189,False,False,0.300321,...,3.9,3.8,2.9,4.0,6.5,6.3,Stable,0.203236,0.060189,0.300321
361,202211,TD,True,0.527107,False,True,0.697625,True,False,0.335496,...,3.5,4.3,3.5,7.3,5.6,3.3,Stable,0.527107,0.302375,0.335496
362,202212,TD,True,0.555677,False,True,0.729246,False,False,0.324000,...,3.5,4.3,3.5,7.3,5.6,3.3,Stable,0.555677,0.729246,0.324000


In [23]:
df_indicators['sowc_women-s-economic-empowerment__unemployment-rate-2010-2020-r_male_rural']

0      9.5
1      9.5
2      8.1
3      NaN
4      1.5
      ... 
213    NaN
214    NaN
215    NaN
216    NaN
217    NaN
Name: sowc_women-s-economic-empowerment__unemployment-rate-2010-2020-r_male_rural, Length: 218, dtype: float64

In [24]:
df_merged['sowc_women-s-economic-empowerment__unemployment-rate-2010-2020-r_male_rural']

0       2.2
1       2.2
2       0.5
3       0.5
4       0.5
       ... 
359    12.6
360    12.6
361     2.7
362     2.7
363     2.7
Name: sowc_women-s-economic-empowerment__unemployment-rate-2010-2020-r_male_rural, Length: 364, dtype: float64

In [25]:
df_merged_copy.columns[df_merged.isnull().sum()<5]

Index(['yearmonth', 'fips', 'y_pred_transformer', 'y_pred_proba_transformer',
       'y_true_transformer', 'y_pred_xgboost', 'y_pred_proba_xgboost',
       'y_true_xgboost', 'y_pred_ffnn', 'y_pred_proba_ffnn',
       ...
       'fsi_p1:_state_legitimacy', 'fsi_p2:_public_services',
       'fsi_p3:_human_rights', 'fsi_c1:_security_apparatus',
       'fsi_c2:_factionalized_elites', 'fsi_x1:_external_intervention',
       'fsi_category', 'transformer_probability_prediction_error',
       'xgboost_probability_prediction_error',
       'ffnn_probability_prediction_error'],
      dtype='object', length=115)

In [26]:
from pandas.api.types import is_numeric_dtype

df_of_pvalues = pd.DataFrame({
    'i': [],
    'T_x': [],
    'T_f': [],
    'x_f': []
})

list_of_pvalue_T_x = np.zeros(1343-14)
list_of_pvalue_T_f = np.zeros(1343-14)
list_of_pvalue_x_f = np.zeros(1343-14)

for i in range(14,1343):
    col = df_merged.iloc[:,i]
    if is_numeric_dtype(col):
        adjustable_condition_lst = df_merged[col>col.mean()]
        if adjustable_condition_lst.shape[0]>0:
            a = stats.mannwhitneyu(adjustable_condition_lst['transformer_probability_prediction_error'], adjustable_condition_lst['xgboost_probability_prediction_error'])[1]
            b = stats.mannwhitneyu(adjustable_condition_lst['transformer_probability_prediction_error'], adjustable_condition_lst['ffnn_probability_prediction_error'])[1] 
            c = stats.mannwhitneyu(adjustable_condition_lst['xgboost_probability_prediction_error'], adjustable_condition_lst['ffnn_probability_prediction_error'])[1]
            df_of_pvalues.loc[len(df_of_pvalues.index)] = [i, a, b, c] 

In [27]:
#this is to see the list of the smallest values.
df_of_pvalues.nsmallest(10, 'T_f')

,i,T_x,T_f,x_f
171,185.0,0.513702,0.000002,9.174826e-06
39,53.0,0.069332,0.000009,3.187419e-08
160,174.0,0.172071,0.000013,1.454253e-06
162,176.0,0.483509,0.000014,1.076825e-05
159,173.0,0.114437,0.000017,1.441254e-06
172,186.0,0.237348,0.000038,1.172135e-05
161,175.0,0.349926,0.000050,8.717458e-06
75,89.0,0.042853,0.000064,5.311670e-07
1311,1332.0,0.136779,0.000067,6.744786e-07
110,124.0,0.453756,0.000068,9.141477e-05


In [28]:
#this is to figure out the name of the column
df_merged.iloc[:, 53]

0       NaN
1       NaN
2      55.1
3      55.1
4      55.1
       ... 
359    32.9
360    32.9
361    58.2
362    58.2
363    58.2
Name: sowc_maternal-and-newborn-health__demand-for-family-planning-satisfied-with-modern-methods-2016-2021-r_service-coverage-sub-index-on-reproductive-maternal-newborn-and-child-health, Length: 364, dtype: float64

In [29]:
#this is just to double check the results
col = df_merged.iloc[:,53]
adjustable_condition_lst = df_merged[col>col.mean()]
a = stats.mannwhitneyu(adjustable_condition_lst['transformer_probability_prediction_error'], adjustable_condition_lst['xgboost_probability_prediction_error'])[1]
b = stats.mannwhitneyu(adjustable_condition_lst['transformer_probability_prediction_error'], adjustable_condition_lst['ffnn_probability_prediction_error'])[1] 
c = stats.mannwhitneyu(adjustable_condition_lst['xgboost_probability_prediction_error'], adjustable_condition_lst['ffnn_probability_prediction_error'])[1]

print(a,b,c)

0.06933238064166371 9.004449636736636e-06 3.187418666551089e-08


# ENDS HERE

In [30]:
np.random.seed(123)

reps = 500
#changable_condition_lst = df_indicators[~(df_indicators.wbi_lending_category == 'IBRD') | ~(df_indicators.wbi_lending_category == 'IDA')]['iso3']
changable_condition_lst = df_indicators[df_indicators['sowc_education__equitable-access_out-of-school-rate-2013-2022-r_upper-secondary-education_male']>28.156608703703704]['iso3']


bootstrapped_difference_between_transformer_xgboost = np.zeros(reps)
bootstrapped_difference_between_transformer_ffnn = np.zeros(reps)
bootstrapped_difference_between_xgboost_ffnn = np.zeros(reps)

sample_difference_transformer_xgboost = df_preds[df_preds['iso3'].isin(changable_condition_lst)]['transformer_probability_prediction_error'] - \
                                                      df_preds[df_preds['iso3'].isin(changable_condition_lst)]['xgboost_probability_prediction_error']

sample_difference_transformer_ffnn = df_preds[df_preds['iso3'].isin(changable_condition_lst)]['transformer_probability_prediction_error'] - \
                                                      df_preds[df_preds['iso3'].isin(changable_condition_lst)]['ffnn_probability_prediction_error']

sample_difference_xgboost_ffnn = df_preds[df_preds['iso3'].isin(changable_condition_lst)]['xgboost_probability_prediction_error'] - \
                                                      df_preds[df_preds['iso3'].isin(changable_condition_lst)]['ffnn_probability_prediction_error']

for rep in range(reps):
  bootstrapped_difference_between_transformer_xgboost[rep] = np.random.choice(sample_difference_transformer_xgboost,
                                                                              size=df_preds['iso3'].isin(changable_condition_lst).shape[0]).mean()
  bootstrapped_difference_between_transformer_ffnn[rep] = np.random.choice(sample_difference_transformer_ffnn,
                                                                           size=df_preds['iso3'].isin(changable_condition_lst).shape[0]).mean()
  bootstrapped_difference_between_xgboost_ffnn[rep] = np.random.choice(sample_difference_xgboost_ffnn,
                                                                       size=df_preds['iso3'].isin(changable_condition_lst).shape[0]).mean()

transformer_xgboost_CI = np.round(np.quantile(bootstrapped_difference_between_transformer_xgboost, [0.025, 0.975]), 3)
transformer_ffnn_CI = np.round(np.quantile(bootstrapped_difference_between_transformer_ffnn, [0.025, 0.975]), 3)
xgboost_ffnn_CI = np.round(np.quantile(bootstrapped_difference_between_xgboost_ffnn, [0.025, 0.975]), 3)

transformer_xgboost_HT =stats.mannwhitneyu(df_preds[df_preds['iso3'].isin(changable_condition_lst)]['transformer_probability_prediction_error'],
                                           df_preds[df_preds['iso3'].isin(changable_condition_lst)]['xgboost_probability_prediction_error'])

transformer_ffnn_HI =stats.mannwhitneyu(df_preds[df_preds['iso3'].isin(changable_condition_lst)]['transformer_probability_prediction_error'],
                                        df_preds[df_preds['iso3'].isin(changable_condition_lst)]['ffnn_probability_prediction_error'])

xgboost_ffnn_HI =stats.mannwhitneyu(df_preds[df_preds['iso3'].isin(changable_condition_lst)]['xgboost_probability_prediction_error'],
                                    df_preds[df_preds['iso3'].isin(changable_condition_lst)]['ffnn_probability_prediction_error'])


print(f"Transformer, xgboost: {transformer_xgboost_CI}, {transformer_xgboost_HT}")
print(f"Transformer, ffnn: {transformer_ffnn_CI}, {transformer_ffnn_HI}")
print(f"xgboost, ffnn: {xgboost_ffnn_CI}, {xgboost_ffnn_HI}")



Transformer, xgboost: [-0.069 -0.023], MannwhitneyuResult(statistic=5524.0, pvalue=0.018547054580071977)
Transformer, ffnn: [-0.035  0.   ], MannwhitneyuResult(statistic=6405.0, pvalue=0.5280832419362722)
xgboost, ffnn: [0.009 0.047], MannwhitneyuResult(statistic=7849.0, pvalue=0.028369999800106408)
